In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot 
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates

In [2]:
pair = "EURUSD"
NB_H1_CANDLES = 2000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 


In [4]:
rates = Rates(pair, 13*NB_H1_CANDLES, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

4e-05

In [6]:
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [7]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an.pivots_l = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an.pivots_h = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)

In [8]:
df_an['perv_pivot_h'] = 0
df_an['perv_pivot_l'] = 0

In [9]:
perv_pivot_l = df_an.low.iloc[0]
perv_pivot_h = df_an.high.iloc[0]

for i in range(len(df_an)):
    if df_an.pivots_h.iloc[i]:
        perv_pivot_h = df_an.high.iloc[i]
    if df_an.pivots_l.iloc[i]:
        perv_pivot_l = df_an.low.iloc[i]
    df_an['perv_pivot_h'].iloc[i] = perv_pivot_h
    df_an['perv_pivot_l'].iloc[i] = perv_pivot_l

C:\Users\meysam\AppData\Local\Temp\ipykernel_6148\1397941026.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['perv_pivot_h'].iloc[i] = perv_pivot_h
C:\Users\meysam\AppData\Local\Temp\ipykernel_6148\1397941026.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['perv_pivot_l'].iloc[i] = perv_pivot_l


In [10]:
df_an['close_perv'] = df_an.close.shift(1)

In [11]:
df_an['BUY_SIG'] = (df_an.close_perv < df_an.perv_pivot_h) & (df_an.close >  df_an.perv_pivot_h)
df_an['SELL_SIG'] = (df_an.close_perv > df_an.perv_pivot_l) & (df_an.close <  df_an.perv_pivot_l)
df_an.SELL_SIG.sum()

106

In [29]:
PROFIT_FACTOR = 1.5
df_an.loc[df_an['BUY_SIG'], 'SL'] = df_an.low_perv-SPREAD
df_an.loc[df_an['BUY_SIG'], 'TP'] = PROFIT_FACTOR*(df_an.close-df_an.SL)+df_an.close

df_an.loc[df_an['SELL_SIG'], 'SL'] = df_an.high_perv+SPREAD
df_an.loc[df_an['SELL_SIG'], 'TP'] = PROFIT_FACTOR*(df_an.close-df_an.SL)+df_an.close


df_an.fillna(0)

,time,open,high,low,close,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next,perv_pivot_h,perv_pivot_l,close_perv,BUY_SIG,SELL_SIG,SL,TP,SIGNAL
0,2022-11-29 10:00:00,1.03813,1.03927,1.03508,1.03831,False,False,0.00000,0.00000,1.03559,1.03872,1.03927,1.03508,0.00000,False,False,0.0,0.0,0.0
1,2022-11-29 11:00:00,1.03832,1.03872,1.03559,1.03651,False,False,1.03508,1.03927,1.03636,1.03936,1.03927,1.03508,1.03831,False,False,0.0,0.0,0.0
2,2022-11-29 12:00:00,1.03649,1.03936,1.03636,1.03827,False,True,1.03559,1.03872,1.03676,1.03861,1.03936,1.03508,1.03651,False,False,0.0,0.0,0.0
3,2022-11-29 13:00:00,1.03826,1.03861,1.03676,1.03826,False,False,1.03636,1.03936,1.03599,1.03839,1.03936,1.03508,1.03827,False,False,0.0,0.0,0.0
4,2022-11-29 14:00:00,1.03825,1.03839,1.03599,1.03755,False,False,1.03676,1.03861,1.03247,1.03846,1.03936,1.03508,1.03826,False,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2023-03-28 14:00:00,1.08228,1.08366,1.08204,1.08313,False,False,1.08143,1.08293,1.08300,1.08468,1.08341,1.08103,1.08228,False,False,0.0,0.0,0.0
1995,2023-03-28 15:00:00,1.08313,1.08468,1.08300,1.08394,False,True,1.08204,1.08366,1.08222,1.08394,1.08468,1.08103,1.08313,False,False,0.0,0.0,0.0
1996,2023-03-28 16:00:00,1.08394,1.08394,1.08222,1.08239,False,False,1.08300,1.08468,1.08164,1.08411,1.08468,1.08103,1.08394,False,False,0.0,0.0,0.0
1997,2023-03-28 17:00:00,1.08241,1.08411,1.08164,1.08321,True,False,1.08222,1.08394,1.08312,1.08460,1.08468,1.08164,1.08239,False,False,0.0,0.0,0.0


In [30]:
BUY = 1
SELL = -1
NONE = 0
df_an.loc[df_an['BUY_SIG'], 'SIGNAL'] = BUY
df_an.loc[df_an['SELL_SIG'], 'SIGNAL'] = SELL
df_an.fillna(0)

,time,open,high,low,close,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next,perv_pivot_h,perv_pivot_l,close_perv,BUY_SIG,SELL_SIG,SL,TP,SIGNAL
0,2022-11-29 10:00:00,1.03813,1.03927,1.03508,1.03831,False,False,0.00000,0.00000,1.03559,1.03872,1.03927,1.03508,0.00000,False,False,0.0,0.0,0.0
1,2022-11-29 11:00:00,1.03832,1.03872,1.03559,1.03651,False,False,1.03508,1.03927,1.03636,1.03936,1.03927,1.03508,1.03831,False,False,0.0,0.0,0.0
2,2022-11-29 12:00:00,1.03649,1.03936,1.03636,1.03827,False,True,1.03559,1.03872,1.03676,1.03861,1.03936,1.03508,1.03651,False,False,0.0,0.0,0.0
3,2022-11-29 13:00:00,1.03826,1.03861,1.03676,1.03826,False,False,1.03636,1.03936,1.03599,1.03839,1.03936,1.03508,1.03827,False,False,0.0,0.0,0.0
4,2022-11-29 14:00:00,1.03825,1.03839,1.03599,1.03755,False,False,1.03676,1.03861,1.03247,1.03846,1.03936,1.03508,1.03826,False,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2023-03-28 14:00:00,1.08228,1.08366,1.08204,1.08313,False,False,1.08143,1.08293,1.08300,1.08468,1.08341,1.08103,1.08228,False,False,0.0,0.0,0.0
1995,2023-03-28 15:00:00,1.08313,1.08468,1.08300,1.08394,False,True,1.08204,1.08366,1.08222,1.08394,1.08468,1.08103,1.08313,False,False,0.0,0.0,0.0
1996,2023-03-28 16:00:00,1.08394,1.08394,1.08222,1.08239,False,False,1.08300,1.08468,1.08164,1.08411,1.08468,1.08103,1.08394,False,False,0.0,0.0,0.0
1997,2023-03-28 17:00:00,1.08241,1.08411,1.08164,1.08321,True,False,1.08222,1.08394,1.08312,1.08460,1.08468,1.08164,1.08239,False,False,0.0,0.0,0.0


In [31]:
df_an

,time,open,high,low,close,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next,perv_pivot_h,perv_pivot_l,close_perv,BUY_SIG,SELL_SIG,SL,TP,SIGNAL
0,2022-11-29 10:00:00,1.03813,1.03927,1.03508,1.03831,False,False,NaN,NaN,1.03559,1.03872,1.03927,1.03508,NaN,False,False,NaN,NaN,NaN
1,2022-11-29 11:00:00,1.03832,1.03872,1.03559,1.03651,False,False,1.03508,1.03927,1.03636,1.03936,1.03927,1.03508,1.03831,False,False,NaN,NaN,NaN
2,2022-11-29 12:00:00,1.03649,1.03936,1.03636,1.03827,False,True,1.03559,1.03872,1.03676,1.03861,1.03936,1.03508,1.03651,False,False,NaN,NaN,NaN
3,2022-11-29 13:00:00,1.03826,1.03861,1.03676,1.03826,False,False,1.03636,1.03936,1.03599,1.03839,1.03936,1.03508,1.03827,False,False,NaN,NaN,NaN
4,2022-11-29 14:00:00,1.03825,1.03839,1.03599,1.03755,False,False,1.03676,1.03861,1.03247,1.03846,1.03936,1.03508,1.03826,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2023-03-28 14:00:00,1.08228,1.08366,1.08204,1.08313,False,False,1.08143,1.08293,1.08300,1.08468,1.08341,1.08103,1.08228,False,False,NaN,NaN,NaN
1995,2023-03-28 15:00:00,1.08313,1.08468,1.08300,1.08394,False,True,1.08204,1.08366,1.08222,1.08394,1.08468,1.08103,1.08313,False,False,NaN,NaN,NaN
1996,2023-03-28 16:00:00,1.08394,1.08394,1.08222,1.08239,False,False,1.08300,1.08468,1.08164,1.08411,1.08468,1.08103,1.08394,False,False,NaN,NaN,NaN
1997,2023-03-28 17:00:00,1.08241,1.08411,1.08164,1.08321,True,False,1.08222,1.08394,1.08312,1.08460,1.08468,1.08164,1.08239,False,False,NaN,NaN,NaN


In [40]:
cp = CandlePlot(df_an, candles=True)

trades_buy = cp.df_plot[cp.df_plot.BUY_SIG]
trades_sell = cp.df_plot[cp.df_plot.SELL_SIG]

cp.fig.add_trace(go.Scatter(
        x = trades_buy.sTime,
        y = trades_buy.close,
        mode = 'markers',
        marker=dict(color='#0000FF', size=5)
    ))

cp.fig.add_trace(go.Scatter(
        x = trades_buy.sTime,
        y = trades_buy.SL,
        mode = 'markers',
        marker=dict(color='#FF0000', size=5)
    ))

cp.fig.add_trace(go.Scatter(
        x = trades_buy.sTime,
        y = trades_buy.TP,
        mode = 'markers',
        marker=dict(color='#00FF00', size=5)
    ))

cp.fig.add_trace(go.Scatter(
        x = trades_sell.sTime,
        y = trades_sell.close,
        mode = 'markers',
        marker=dict(color='#0000FF', size=5)
    ))

cp.fig.add_trace(go.Scatter(
        x = trades_sell.sTime,
        y = trades_sell.SL,
        mode = 'markers',
        marker=dict(color='#FF0000', size=5)
    ))

cp.fig.add_trace(go.Scatter(
        x = trades_sell.sTime,
        y = trades_sell.TP,
        mode = 'markers',
        marker=dict(color='#00FF00', size=5)
    ))

# cp.fig.add_trace(go.Scatter(
#         x = trades_sell.sTime,
#         y = trades_sell.close,
#         mode = 'markers',
#         marker=dict(color='#0000FF', size=10)
#     ))
cp.fig.add_trace(go.Scatter(
                x=cp.df_plot.sTime,
                y=cp.df_plot['perv_pivot_h'],
                line=dict(width=1),
                name='perv_pivot_h'
            ))
cp.fig.add_trace(go.Scatter(
                x=cp.df_plot.sTime,
                y=cp.df_plot['perv_pivot_l'],
                line=dict(width=1),
                name='perv_pivot_l'
            ))

cp.show_plot()

In [37]:
gt = GuruTester2(
        df_an,
        df_m5,
        SPREAD,
        PROFIT_FACTOR = PROFIT_FACTOR,
        use_spread=True
    )

prepare_data...


In [38]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.3827751196172249
-1.0    129
 1.5     80
Name: result, dtype: int64


In [39]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 1.5):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_6148\817891233.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])